# Preprocessing 

In [1]:
import docx
import pandas as pd
import glob
import os
from pathlib import Path
import re
import pickle
import string
import numpy as np
# pip install germalemma

## Loading files into the df and extracting the names and year

In [4]:
dir_path = "../data/rel_POL/"

# Get a list of all files in the directory
file_list = os.listdir(dir_path)

# Filter the list to include only CSV files
csv_files = [f for f in file_list if f.endswith('.csv')]

# Read each CSV file into a DataFrame, add a 'filename' column, and store them in a list
dfs = []
for csv_file in csv_files:
    csv_path = os.path.join(dir_path, csv_file)
    df = pd.read_csv(csv_path)
    df['filename'] = csv_file  # Add filename as a new column
    dfs.append(df)

# Concatenate all DataFrames in the list into a single DataFrame
df = pd.concat(dfs, ignore_index=True)

In [9]:
# doc = docx.Document("../data/rel_POL/PL_ORTHO_press_39.docx")
# # Create an empty string to store the text
# text = ''

# for para in doc.paragraphs:
#     # Add the text to the string
#     text += para.text.strip() + ' '

In [10]:
df.head(2)

,title,txt,auth,added,edition,tags,links,filename,Unnamed: 0,leads
0,Kościół w pandemii,"Z danych statystycznych za 2020 r. wynika, że ...",BOGUMIŁ ŁOZIŃSKI,DODANE 16.12.2021 00:15,GN 50/2021,TAGI:\ngość niedzielny temat gościa,https://www.gosc.pl/doc/7271155.Kosciol-w-pand...,gosc_kościół pandemia.csv,NaN,NaN
1,Kościół po pandemii,Skierowanie w kierunku misji czyli wyjście na ...,NaN,DODANE 27.09.2020 09:30,KAI,TAGI:\nabp stanisław gądecki biskupi europy cc...,https://www.gosc.pl/doc/6543819.Kosciol-po-pan...,gosc_kościół pandemia.csv,NaN,NaN


In [ ]:
df['len_text'] = df['txt_1'].apply(lambda x: len(x.split()))

In [84]:
# sprawdzam jak długi jest tekst i czy czegoś nie ucięło, wyrywkowo
df['len_text'].describe()

count      797.000000
mean       706.422836
std       1196.333611
min        120.000000
25%        320.000000
50%        446.000000
75%        778.000000
max      29425.000000
Name: len_text, dtype: float64

In [85]:
min_row = df.loc[df['len_text'].idxmin()]

# Print the row
print(min_row)

date                                         2021-07-01 00:00:00
year                                         2021-01-01 00:00:00
f_name         ../data/rel_DEU/IslamischeZeitung/IZ_RandomSam...
raw_txt        *** Date of publication: 17/07/2021 Link: http...
txt_1          Jakarta (KNA). Indonesien hat in dieser Woche ...
len_text                                                     120
cleaned_txt    Jakarta KNA Indonesien hat in dieser Woche Ind...
Name: 106, dtype: object


## Klasyfikacja do denominacji

In [94]:
dict = {
        "Bistum" : 'Cath_Ch', "BR" : 'Cath_Ch', "DBK" : 'Cath_Ch', 'chrismon': 'Cath_Ch', 'katholisch.de': 'Cath_Ch',
        
        "ELKB" : 'Protest', "EKD" : 'Protest', "EVLKS" : 'Protest',

        "AGiD" : 'Anthrop', "Ant" : 'Anthrop',

        "IRD" : 'Muslim', "KRM" : 'Muslim', "ZDM" : 'Muslim', "DITIB" : 'Muslim', "VIKZ" : 'Muslim', "IRD" : 'Muslim', "ZMD" : 'Muslim', "Islamrat": 'Muslim', "IslamischeZeitung": "IRD"
    }

        

In [95]:
for key, value in dict.items():
    df.loc[df['f_name'].str.contains(key), 'Denomination'] = value


In [96]:
df[df['Denomination'].isna()]

,date,year,f_name,raw_txt,txt_1,len_text,cleaned_txt,Denomination


In [12]:
df

,title,txt,auth,added,edition,tags,links,filename,Unnamed: 0,leads
0,Kościół w pandemii,"Z danych statystycznych za 2020 r. wynika, że ...",BOGUMIŁ ŁOZIŃSKI,DODANE 16.12.2021 00:15,GN 50/2021,TAGI:\ngość niedzielny temat gościa,https://www.gosc.pl/doc/7271155.Kosciol-w-pand...,gosc_kościół pandemia.csv,NaN,NaN
1,Kościół po pandemii,Skierowanie w kierunku misji czyli wyjście na ...,NaN,DODANE 27.09.2020 09:30,KAI,TAGI:\nabp stanisław gądecki biskupi europy cc...,https://www.gosc.pl/doc/6543819.Kosciol-po-pan...,gosc_kościół pandemia.csv,NaN,NaN
2,Adwent w czasach pandemii,"Biskupi zachęcają księży, by w tym trudnym cza...",PIE,DODANE 26.11.2020 16:49,NaN,NaN,https://www.gosc.pl/doc/6628248.Adwent-w-czasa...,gosc_kościół pandemia.csv,NaN,NaN
3,Hiszpania: Kościół modli się za ofiary pandemii,Data nie jest przypadkowa. 26 lipca przypada l...,NaN,DODANE 26.07.2020 19:00,VATICANNEWS.VA,NaN,https://www.gosc.pl/doc/6425668.Hiszpania-Kosc...,gosc_kościół pandemia.csv,NaN,NaN
4,Jak będzie wyglądał Kościół w Polsce po pandemii,"""W Polsce ciągle mamy Kościół mocno klerykalny...",NaN,DODANE 01.06.2021 09:50,KAI,TAGI:\nabp grzegorz ryś archidiecezja łódzka e...,https://www.gosc.pl/doc/6896505.Jak-bedzie-wyg...,gosc_kościół pandemia.csv,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2384,Przeniesienie Biura Oddziału we Włoszech posuw...,Mimo kryzysu ekonomicznego i zdrowotnego przen...,NaN,16 LISTOPADA 2020\nWŁOCHY,NaN,NaN,https://www.jw.org/open?docid=702020456&prefer...,jw_jehowy+%2Bpandemia.csv,NaN,NaN
2385,10 lat istnienia serwisu internetowego jw.org ...,"Jak już informowaliśmy, w 2022 roku minęło 25 ...",NaN,28 GRUDNIA 2022\nWIADOMOŚCI ZE ŚWIATA,NaN,NaN,https://www.jw.org/open?docid=702022287&prefer...,jw_jehowy+%2Bpandemia.csv,NaN,NaN
2386,Czy istnieje środek zaradczy?,Czy istnieje środek zaradczy?\n„NABIERAMY cora...,NaN,NaN,NaN,NaN,https://wol.jw.org/wol/finder?wtlocale=P&docid...,jw_jehowy+%2Bpandemia.csv,NaN,NaN
2387,Gdzie AIDS jest pandemią,Gdzie AIDS jest pandemią\nW CIĄGU niespełna 15...,NaN,NaN,NaN,NaN,https://wol.jw.org/wol/finder?wtlocale=P&docid...,jw_jehowy+%2Bpandemia.csv,NaN,NaN


In [11]:
df.to_csv('../data/clean/rel_pol_clean.csv')

In [2]:
from germalemma import GermaLemma

lemmatizer = GermaLemma()

# passing the word and the POS tag ("N" for noun)
lemma = lemmatizer.find_lemma('Feinstaubbelastungen', 'N')
print(lemma)
# -> lemma is "Feinstaubbelastung"

Feinstaubbelastung
